In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# %matplotlib widget
basePath = "./sims.TNG/TNG50-1/output"
from tqdm import tqdm
import gc
import os

Snapshot_Init = 71
Snapshot_End = 99
Lbsg_Indcies_Array = np.array(
    [117251, 300903, 324123, 329508, 382215, 400973, 419618, 457431, 460722, 490814]
)

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3


In [7]:
Current_Snapshot=99
Subhalo_Index=329508


In [3]:
partType4_Fields=['Coordinates','Masses']


In [8]:
partType4_dict=il.snapshot.loadSubhalo(basePath,Current_Snapshot,Subhalo_Index,4,partType4_Fields)
subhalo_dict=il.groupcat.loadSingle(basePath,Current_Snapshot,Subhalo_Index)
